In [2]:
import scipy.misc
from skimage import io
from skimage.color import rgb2hsv
import skimage
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import os

### Statistics Collection for Russian Images
- data includes images that are capable of providing specific statistics
- images unable to provide statistics are stored in separate list

In [85]:
GENRE_URL = 'https://www.wikiart.org/en/artists-by-genre/portrait/text-list'
ARTIST_URL = 'https://www.wikiart.org/{artist}/all-works/text-list'
PAINTING_URL = 'https://www.wikiart.org{painting_path}'

In [86]:
request = requests.get(GENRE_URL)
soup = BeautifulSoup(request.text,'lxml')

In [24]:
artist_div =  soup.find('div', class_ = "masonry-text-view masonry-text-view-all")

In [87]:
#Finding only Russian artists
artist_tup = []
artists = []
for link in artist_div.find_all('li'):
#     if link.span.text[2:] == 'Russian' or link.span.text[2:] == 'French':
    if link.span.text[2:] == 'Russian':
#         print(link.a['href'][4:])
        artists.append(link.a['href'])
        artist_tup.append((link.a['href'][4:],link.span.text[2:]))
#         print(link.span.text[2:])

In [88]:
# Formating links to each artist
for x in range(len(artists)):
    artists[x] = ARTIST_URL.format(artist = artists[x])

In [90]:
#Gathers all images from individual artists
def image_scrape(lst):
    painting_paths = []
    for x in lst:
        #Gathering links of artist page
        artist_page = requests.get(x)
        soup = BeautifulSoup(artist_page.text, 'lxml')
        # retreive all rows in painting-list
        for li in soup.find_all('li', {'class': 'painting-list-text-row'}):
            # retrieve all links in the current row
            for link in li.find_all('a'):
                href = link.get('href')
                # store in dictionary
                painting_paths.append(href)
    return painting_paths

In [91]:
#May take couple minutes to run 
painting_lst = image_scrape(artists)

In [92]:
#Saving painting name, date, and nationality for the paintings

df = pd.DataFrame(columns=['name','year','nationality'])
def name_and_date(painting_url):
    for x in range(len(artist_tup)):
        if artist_tup[x][0] in painting_url:
            try:
                year = int(painting_url[-4:])
                name = painting_url[5+ len(artist_tup[x][0]):]
                nationality = artist_tup[x][1]
                
                df.loc[len(df)] = [name,year,nationality]
            except:
                year = np.nan
                name = painting_url[5+ len(artist_tup[x][0]):]
                nationality = artist_tup[x][1]
                
                df.loc[len(df)] = [name,year,nationality]

In [93]:
for path in painting_lst:
    name_and_date(path)

In [95]:
image_direc = "C://Users//Kangster150//Downloads//WD HDD//DSC 160 Images//image_dir//All Russian Photos"
image_collection = []
for i in os.listdir(image_direc):
    image_file = os.path.join(image_direc, i)
    no_jpg = i[:-4]
    if no_jpg in df['name'].tolist():
        image_collection.append(i)

In [96]:
def calc_stats(filename, file_folder):
    image_stats = []
    
    #file path for file name created
    image_direc = "C://Users//Kangster150//Downloads//WD HDD//DSC 160 Images//image_dir//" + str(file_folder)
    image_file = os.path.join(image_direc, filename)
    
    # collection of width and height
    curr_file = io.imread(image_file)
    image_width = curr_file.shape[1]
    image_height = curr_file.shape[0]
    
    # collection of (mean) hue, saturation and value
    image_rgb = curr_file
    image_hsv = rgb2hsv(image_rgb)
    
    image_hue = image_hsv[:, :, 0]
    image_sat = image_hsv[:,:, 1]
    image_val = image_hsv[:, :, 2]
    
    hue_mean = np.mean(image_hue, axis=(0,1))
    sat_mean = np.mean(image_sat, axis=(0,1))
    val_mean = np.mean(image_val)
    
    image_stats.append(image_width)
    image_stats.append(image_height)
    image_stats.append(hue_mean)
    image_stats.append(sat_mean)
    image_stats.append(val_mean)
    
    return image_stats

In [99]:
# your code here
russian_stats_df = pd.DataFrame(columns=['name', 'width', 'height', 'mean_hue',
                                       'mean_saturation', 'mean_brightness'])
no_russian_stats = []
for i in image_collection:
    try:
        curr_img_stats = calc_stats(i, "All Russian Photos")
        russian_stats_df = russian_stats_df.append({'name':i[:-4], 'width': curr_img_stats[0], 
                                                  'height': curr_img_stats[1] , 
                                                  'mean_hue': curr_img_stats[2],
                                                   'mean_saturation': curr_img_stats[3], 
                                                  'mean_brightness': curr_img_stats[4]} , ignore_index=True)
    except:
        no_russian_stats.append(i)

In [101]:
#Function to normalize naming
def remove_date(string):
    if string[-4:].isnumeric() == True:
        return string[:-5]
    else:
        return string

In [102]:
russian_stats_df['name'] = russian_stats_df['name'].apply(remove_date)

In [105]:
df['name'] = df['name'].apply(remove_date)

In [110]:
russian_complete = pd.merge(df,russian_stats_df, on='name')

### Statistics Collection for French Images
- data includes images that are capable of providing specific statistics
- images unable to provide statistics are stored in separate list

In [111]:
#Finding only French artists
artist_tup = []
artists = []
for link in artist_div.find_all('li'):
#     if link.span.text[2:] == 'Russian' or link.span.text[2:] == 'French':
    if link.span.text[2:] == 'French':
#         print(link.a['href'][4:])
        artists.append(link.a['href'])
        artist_tup.append((link.a['href'][4:],link.span.text[2:]))
#         print(link.span.text[2:])

In [112]:
# Formating links to each artist
for x in range(len(artists)):
    artists[x] = ARTIST_URL.format(artist = artists[x])

In [113]:
#Gathers all images from individual artists
def image_scrape(lst):
    painting_paths = []
    for x in lst:
        #Gathering links of artist page
        artist_page = requests.get(x)
        soup = BeautifulSoup(artist_page.text, 'lxml')
        # retreive all rows in painting-list
        for li in soup.find_all('li', {'class': 'painting-list-text-row'}):
            # retrieve all links in the current row
            for link in li.find_all('a'):
                href = link.get('href')
                # store in dictionary
                painting_paths.append(href)
    return painting_paths

In [114]:
#May take couple minutes to run 
painting_lst = image_scrape(artists)

In [116]:
for path in painting_lst:
    name_and_date(path)

In [117]:
image_direc = "C://Users//Kangster150//Downloads//WD HDD//DSC 160 Images//image_dir//All French Photos"
image_collection = []
for i in os.listdir(image_direc):
    image_file = os.path.join(image_direc, i)
    no_jpg = i[:-4]
    if no_jpg in df['name'].tolist():
        image_collection.append(i)

In [123]:
# your code here
french_stats_df = pd.DataFrame(columns=['name', 'width', 'height', 'mean_hue',
                                       'mean_saturation', 'mean_brightness'])
no_french_stats = []
for i in image_collection:
    try:
        curr_img_stats = calc_stats(i, "All French Photos")
        french_stats_df = french_stats_df.append({'name':i[:-4], 'width': curr_img_stats[0], 
                                                  'height': curr_img_stats[1] , 
                                                  'mean_hue': curr_img_stats[2],
                                                   'mean_saturation': curr_img_stats[3], 
                                                  'mean_brightness': curr_img_stats[4]} , ignore_index=True)
    except:
        no_french_stats.append(i)

In [124]:
french_stats_df['name'] = french_stats_df['name'].apply(remove_date)

In [125]:
df['name'] = df['name'].apply(remove_date)

In [126]:
french_complete = pd.merge(df,french_stats_df, on='name')

### Complete Image Statistics

In [127]:
complete_image_stats = pd.concat([russian_complete, 
                                  french_complete], ignore_index=True)

In [128]:
complete_image_stats

,name,year,nationality,width,height,mean_hue,mean_saturation,mean_brightness
0,tzarina-natalia-alekseevna-1,NaN,Russian,550,689,0.213050,0.433988,0.237776
1,portrait-of-elizaveta-petrovna-as-a-child,1713,Russian,550,698,0.109722,0.433142,0.232679
2,tzarina-praskovia-ioanovna,1714,Russian,1400,1817,0.250579,0.296722,0.240346
3,peter-i,NaN,Russian,590,800,0.461972,0.368354,0.385637
4,peter-i,NaN,Russian,640,783,0.556476,0.444790,0.135088
...,...,...,...,...,...,...,...,...
3615,view-from-rue-de-la-chaise,2005,French,400,329,0.291260,0.240882,0.420620
3616,view-over-rue-de-grenelle,2005,French,309,413,0.000000,0.000000,0.697255
3617,nightgown,2006,French,400,329,0.251990,0.167029,0.439734
3618,self-portrait-drawing,2008,French,436,600,0.100010,0.106837,0.802831
